## Procesamiento de datos meteorológicos
Este código procesa los datos meteorológicos de AEMET de cada dia del año y los almacena en un archivo CSV en una carpeta llamada 'ClimaData'. Para hacer esto, carga el archivo CSV que contiene información de incendios en España en 2021 y selecciona las fechas correspondientes a los incendios. Luego, crea un conjunto de procesos paralelos utilizando Multiprocessing para procesar los datos meteorológicos correspondientes a cada fecha en el conjunto de fechas de los incendios.

Dentro de la función process_date(), se carga el archivo de datos meteorológicos correspondiente a la fecha proporcionada y se seleccionan las filas que contienen los datos relevantes. Luego, se agregan dos columnas para las coordenadas de latitud y longitud y se carga el archivo con información de la posicion de las estaciones meteorológicas. Se realiza una iteración para encontrar la correspondencia entre el nombre de la estación y los datos meteorológicos. Cuando se encuentra la estación correspondiente, se calculan sus coordenadas a partir de los datos de latitud y longitud que se encuentran en el archivo de información de estaciones. Después de esto, se eliminan las filas que contienen datos faltantes y se renombran las columnas para que tengan nombres más claros. Finalmente, se guarda el conjunto de datos resultante en un archivo CSV en la carpeta 'ClimaData'.

### Resultado
El resultado de este proceso es un conjunto de datos CSV que contiene información meteorológica de toda España para cada fecha en la que se produjo un incendio en 2021

In [ ]:
import multiprocessing as mp
import pandas as pd
import numpy as np

def process_date(date):
    
    print(f"Processing date {date}")

    # Se crea el nombre del archivo que contiene los datos meteorológicos de la fecha a procesar
    file_name = f'Aemet2021-{date[5:]}.csv'
    print(f"Making {file_name}")

    # Se carga el archivo correspondiente a la fecha a procesar y se seleccionan las filas que contienen los datos
    path = f'./ClimaAemet2021/Aemet2021-{date[5:7]}/{file_name}'
    dataset_c = pd.read_excel(path).iloc[4:, :]

    # Se añaden dos columnas al dataset_c para las coordenadas de latitud y longitud
    dataset_c[['latitude', 'longitude']] = np.zeros((len(dataset_c), 2))

    # Se carga el archivo con la información de las estaciones y se itera sobre cada fila de dataset_c buscando las correspondencias
    # entre el nombre de la estación y los datos meteorológicos. Cuando se encuentra la estación correspondiente, se calculan
    # sus coordenadas a partir de los datos de latitud y longitud que se encuentran en el archivo de información de estaciones.
    dataset_e = pd.read_excel('./ListadoEstaciones-20190206.xlsx')
    for i, row in dataset_c.iterrows():
        for j, estacion in dataset_e.iterrows():
            if row[0].upper() == estacion[2].upper():
                sign = -1 if estacion[5][-1] == 'W' else 1
                lat = int(estacion[4][:2]) + int(estacion[4][2:4])/60 + int(estacion[4][4:6])/3600
                long = (int(estacion[5][:2]) + int(estacion[5][2:4])/60 + int(estacion[5][4:6])/3600) * sign
                dataset_c.loc[i, ['latitude', 'longitude']] = [lat, long]
                break

    # Se eliminan las filas que contienen datos faltantes y se renombran las columnas para que tengan nombres más claros
    dataset_c = dataset_c.dropna()
    dataset_c.columns = ['Estación', 'Provincia', 'Temperatura máxima (ºC)', 'Temperatura mínima (ºC)', 'Temperatura media (ºC)', 'Racha (km/h)', 
    'Velocidad máxima (km/h)', 'Precipitación 00-24h (mm)','Precipitación 00-06h (mm)', 'Precipitación 06-12h (mm)', 'Precipitación 12-18h (mm)' ,'Precipitación 18-24h (mm)', 'Latitude', 'Longitude']

    # Se guarda el dataset_c en un archivo CSV en la carpeta 'ClimaData'
    dataset_c.to_csv(f'ClimaData/{file_name}', index=False)

if __name__ == '__main__':
    # Se carga el archivo con la información de los incendios y se seleccionan las columnas necesarias
    dataset_i = pd.read_csv('./modis_2021_Spain_incendios.csv').iloc[:, [0, 1, 2, 5]]
    with mp.Pool(10) as pool:
        pool.map(process_date, dataset_i.iloc[:, 3])

## INTERPOLACION
El objetivo de este código es crear un mapa de calor de España de los datos climáticos correspondientes a una fecha determinada. El código lee un archivo CSV que contiene datos de latitud, longitud y la columna seleccionada, filtra los datos para mostrar solo los ubicados dentro de España y define los límites del mapa de calor. Luego, el código crea una cuadrícula de puntos de latitud y longitud, extrae los puntos y valores correspondientes de la columna elegida y los interpola en los puntos de la cuadrícula utilizando el método de interpolación lineal. Finalmente, el código grafica un mapa de calor utilizando los puntos y valores interpolados y ajusta la posición y los detalles de la visualización. La función "view_heatmap" permite cambiar la columna seleccionada y la fecha de los datos a visualizar.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

def view_heatmap(fecha='2021-01-02', colum='Temperatura media (ºC)'):
    # Lee los datos climáticos de un archivo CSV correspondiente a la fecha seleccionada.
    df = pd.read_csv(f'./ClimaData/Aemet{fecha}.csv')
    # Selecciona las columnas de latitud, longitud y la columna elegida.
    df = df[['Longitude', 'Latitude', colum]]
    # Filtra los datos para que solo muestre los ubicados dentro de España.
    df = df.loc[(df['Latitude'] > 30) & (df['Latitude'] < 44) & (df['Longitude'] > -20)]
    # Si la columna seleccionada es "Velocidad máxima (km/h)", extrae los números de la cadena y los convierte en flotantes.
    if colum == 'Velocidad máxima (km/h)':
        df[colum] = df[colum].str.extract('(\d+)', expand=False).astype(float)
    else:
        df[colum] = df[colum].astype(float)

    # Define los límites del mapa de calor.
    minx, maxx = df['Longitude'].min(), df['Longitude'].max()
    miny, maxy = df['Latitude'].min(), df['Latitude'].max()
    # Crea una cuadrícula de 300x300 puntos de latitud y longitud.
    grdi_x = np.linspace(minx, maxx, num=300, endpoint=False)
    grdi_y = np.linspace(miny, maxy, num=300, endpoint=False)
    # Crea una cuadrícula de latitud y longitud 2D.
    yg, xg = np.meshgrid(grdi_y, grdi_x, indexing='ij')
    # Aplana las cuadrículas para poder usarlas con la función griddata.
    x_g, y_g = xg.ravel(), yg.ravel()

    # Extrae los puntos y los valores correspondientes de la columna elegida.
    points = df[['Longitude', 'Latitude']].values
    values = df[colum].values
    # Interpola los valores en los puntos de la cuadrícula de latitud y longitud 2D utilizando el método de interpolación lineal.
    grid_z0 = griddata(points, values, (x_g, y_g), method='linear')

    # Grafica un mapa de calor utilizando los puntos y valores de la cuadrícula interpolada.
    plt.scatter(x_g, y_g, s=40, marker='s', c=grid_z0, cmap=plt.cm.hsv)
    # Establece las etiquetas de los ejes y la barra de colores.
    plt.ylabel('Latitude')
    plt.xlabel('Longitude')
    plt.colorbar().set_label(colum)
    # Ajusta la posición del mapa de calor.
    plt.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1, wspace=0.2, hspace=0.2)
    # Establece el color de fondo del mapa de calor en negro.
    plt.gca().set_facecolor('xkcd:black')
    plt.show()

In [ ]:
view_heatmap(colum = 'Velocidad máxima (km/h)')
view_heatmap(colum = 'Precipitación 00-24h (mm)')
view_heatmap(colum = 'Temperatura media (ºC)')

## Interpolación de datos climáticos para cada incendio
Este código tiene como objetivo obtener información climática para puntos de incendios en España a partir de un conjunto de datos que incluye la latitud y longitud de los puntos de incendio, así como su fecha de detección. El código carga un conjunto de datos que combina datos de incendios y datos climáticos. Luego, filtra los datos de latitud para excluir aquellos que son menores a 30. Después, agrega nuevas columnas para almacenar los datos climáticos interpolados y guarda el conjunto de datos modificado en un archivo csv.

La función interpolate_climate_data() se utiliza para interpolar los datos climáticos para cada punto de incendio en el conjunto de datos. La función carga los datos climáticos correspondientes a la fecha de detección del incendio y los filtra por límites de latitud y longitud. Luego, extrae la latitud, longitud y valores de datos climáticos, los convierte a valores flotantes y luego interpola los datos climáticos para la ubicación del punto de incendio utilizando el método de interpolación lineal. Finalmente, guarda los datos interpolados en la columna correspondiente del conjunto de datos de incendios.

El código itera sobre cada punto de datos en el conjunto de datos de incendios y llama a la función interpolate_climate_data() para interpolar los datos climáticos para cada columna de datos climáticos. Por último, guarda el conjunto de datos final con los datos climáticos interpolados en un archivo csv.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# Load dataset of fires with climate data
dataset_incendios = pd.read_csv("./modis_2021_Spain_incendios_with_clima.csv")

# Filter out data points with latitude less than 30
dataset_incendios = dataset_incendios[dataset_incendios['latitude'] > 30]

# Add new columns to store interpolated climate data
climate_cols = ['Temperatura máxima (ºC)', 'Temperatura mínima (ºC)', 'Temperatura media (ºC)', 
                'Racha (km/h)', 'Velocidad máxima (km/h)', 'Precipitación 00-24h (mm)', 
                'Precipitación 00-06h (mm)', 'Precipitación 06-12h (mm)', 'Precipitación 12-18h (mm)', 
                'Precipitación 18-24h (mm)']
for col in climate_cols:
    dataset_incendios[col] = np.zeros(len(dataset_incendios))

# Save the modified dataset to disk
dataset_incendios.to_csv("./modis_2021_Spain_incendios_with_clima.csv", index=False)

def interpolate_climate_data(index, column):
    # Load climate data for the date of the current fire data point
    date = dataset_incendios.iloc[index]['acq_date']
    climate_data = pd.read_csv(f'./ClimaData/Aemet{date}.csv')

    # Filter climate data by latitude and longitude bounds
    climate_data = climate_data[(climate_data['Latitude'] > 30) & (climate_data['Latitude'] < 44) & (climate_data['Longitude'] > -20)]

    # Convert climate data values to float
    if str(climate_data.iloc[1][column]).find(' ') != -1:
        for i in range(len(climate_data)):
            climate_data.iloc[i][column] = float(climate_data.iloc[i][column][:climate_data.iloc[i][column].find(' ')])

    # Extract latitude, longitude, and climate data values
    lat = climate_data['Latitude']
    lon = climate_data['Longitude']
    values = np.array(climate_data[column], dtype='float64')
    lat_arr = np.array(lat, dtype='float64')
    lon_arr = np.array(lon, dtype='float64')

    # Interpolate climate data value at the location of the current fire data point
    est_value = griddata((lat_arr, lon_arr), values, (dataset_incendios.iloc[index]['latitude'], dataset_incendios.iloc[index]['longitude']), method='linear')

    # Save the interpolated climate data value to the corresponding column in the fire dataset
    dataset_incendios.loc[index, column] = est_value

    print(f"{column} : {est_value}")

# Iterate over each data point in the fire dataset and interpolate climate data for each column
for index in range(len(dataset_incendios)):
    print(f"{index} of : {len(dataset_incendios)}")
    for column in climate_cols:
        interpolate_climate_data(index, column)

# Save the final fire dataset with interpolated climate data to disk
dataset_incendios.to_csv("./incendios.csv", index=False)

## Datos sin incendio en puntos aleatorios
Este código tiene como objetivo generar un conjunto de datos de puntos aleatorios que representan ubicaciones geográficas y agregar información meteorológica a estos puntos interpolando valores de un archivo de datos meteorológicos para cada punto.

Primero, la función "getDays" crea una lista de fechas desde el 1 de enero de 2021 hasta el 31 de diciembre de 2021. Luego, se crea un DataFrame vacío "datasetRandomPoints" con columnas que incluyen información geográfica y meteorológica.

Luego, se generan 10 puntos aleatorios para cada fecha, y se agregan al DataFrame "datasetRandomPoints".

Después, se itera sobre el DataFrame y se completa la información meteorológica para cada punto utilizando los datos meteorológicos de un archivo CSV correspondiente a la fecha de cada punto.

El código utiliza la función "griddata" de la biblioteca "scipy.interpolate" para interpolar los valores de temperatura para cada punto en "datasetRandomPoints" utilizando los valores de temperatura de los datos meteorológicos del archivo CSV correspondiente a cada punto.

Finalmente, el DataFrame "datasetRandomPoints" se guarda como un archivo CSV sin índice.

In [ ]:
import pandas as pd
import numpy as np
import random
from scipy.interpolate import griddata
from datetime import date, timedelta

# Función para generar una lista de fechas
def getDays():
    # Definimos el rango de fechas
    start_date = date(2021, 1, 1)
    end_date = date(2021, 12, 31)
    delta = timedelta(days=1)
    days = []

    # Generamos una lista con todas las fechas entre el rango definido
    while start_date <= end_date:
        days.append(start_date.strftime("%Y-%m-%d"))
        start_date += delta

    return days

columns = [
'Latitude',
'Longitude',
'acq_date',
'Temperatura máxima (ºC)',
'Temperatura mínima (ºC)',
'Temperatura media (ºC)',
'Racha (km/h)',
'Velocidad máxima (km/h)',
'Precipitación 00-24h (mm)',
'Precipitación 00-06h (mm)',
'Precipitación 06-12h (mm)',
'Precipitación 12-18h (mm)',
'Precipitación 18-24h (mm)']

datasetRandomPoints = pd.DataFrame(columns=columns)

days = getDays()

# Generar 10 puntos aleatorios para cada fecha y añadirlos al dataframe
for day in days:
    for _ in range(10):
        lat = random.uniform(37, 43)
        lon = random.uniform(-6.5, 0)
        datasetRandomPoints = datasetRandomPoints.append({'Latitude': lat, 'Longitude': lon, 'acq_date': day}, ignore_index=True)

# Iterar sobre el dataframe y completar los valores de las variables meteorológicas
for index in range(len(datasetRandomPoints)):
    # Obtener la fecha y la variable meteorológica a completar
    day = datasetRandomPoints.iloc[index, 2]
    for colum in columns[3:]:
        # Leer el archivo CSV correspondiente
        df = pd.read_csv(f'./ClimaData/Aemet{day}.csv')
        # Obtener el índice de la columna de la variable meteorológica
        columnIndex = df.columns.get_loc(colum)
        # Seleccionar las columnas de latitud, longitud y la variable meteorológica de interés
        df = df.iloc[:,[-1,-2,columnIndex]]
        # Filtrar las filas que se encuentran dentro de los límites geográficos
        df = df.query('Latitude > 30 and Latitude < 44 and Longitude > -20')
        # Si la columna contiene texto, extraer el valor numérico
        if str(df.iloc[1,2]).find(' ') != -1:
            for _ in range(len(df)):
                df.iloc[_,2] = float(df.iloc[_,2][:df.iloc[_,2].find(' ')])
        # Separar el dataframe en dos: uno con las columnas de latitud y longitud, y otro con la variable meteorológica de interés
        df2 = df.drop([colum], 1)
        df3 = df[colum]
       # Extraer las columnas 'Latitude' y 'Longitude' del DataFrame de Aemet y almacenarlos en las variables 'lat' y 'lon'
        lat = df2['Latitude']
        lon = df2['Longitude']

        # Extraer los valores de temperatura de la columna especificada en 'colum' del DataFrame de Aemet y almacenarlos en 'values'
        values = np.array(df3, dtype='float64')

        # Convertir 'lat' y 'lon' en arreglos de tipo float64
        arraylat = np.array(lat, dtype='float64')
        arraylon = np.array(lon, dtype='float64')

        # Interpolar los valores de temperatura en el punto aleatorio del DataFrame de 'datasetRandomPoints' utilizando 'griddata' de la biblioteca 'scipy.interpolate'
        est_u = griddata((arraylat, arraylon), values, (datasetRandomPoints.iloc[index,0], datasetRandomPoints.iloc[index,1]), method='linear')

        # Obtener el índice de la columna de 'colum' en 'datasetRandomPoints'
        columnIndex = datasetRandomPoints.columns.get_loc(colum)

        # Almacenar el valor interpolado en la columna correspondiente de 'datasetRandomPoints'
        datasetRandomPoints.iloc[index, columnIndex] = est_u

# Guardar 'datasetRandomPoints' como un archivo CSV sin el índice
datasetRandomPoints.to_csv('datasetRandomPoints.csv', index=False)

## Dataset combinado de incendios y puntos aleatorios sin incendios para entrenamiento de red neuronal
Este código tiene como objetivo combinar dos conjuntos de datos: uno que contiene información sobre incendios y otro que contiene información sobre puntos aleatorios donde no los hubo. El objetivo final es crear un único conjunto de datos que contenga información tanto sobre los incendios como sobre los puntos aleatorios y utilizarlo para entrenar modelos de aprendizaje automático para predecir la ocurrencia de incendios.

En el código, se lee y se seleccionan las columnas relevantes de ambos conjuntos de datos, se extraen el mes, día y año de la fecha en ambos conjuntos de datos, se elimina la columna "acq_date" en ambos conjuntos de datos y se agrega una columna "fire" en cada conjunto de datos para indicar si es un incendio o no. Luego, se concatenan ambos conjuntos de datos en uno solo y se guarda como archivo csv.

In [ ]:
import pandas as pd

# Lectura de los datasets
datasetRandomPoints = pd.read_csv("./datasetRandomPoints.csv")
datasetIncendios = pd.read_csv("./incendios.csv")

# Seleccionar columnas relevantes del dataset de incendios
datasetIncendios = datasetIncendios.iloc[:, [0, 1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]

# Extraer mes, día y año de las fechas en ambos datasets
datasetRandomPoints['month'] = datasetRandomPoints['acq_date'].apply(lambda x: int(x[5:7]))
datasetRandomPoints['day'] = datasetRandomPoints['acq_date'].apply(lambda x: int(x[8:10]))
datasetRandomPoints['year'] = datasetRandomPoints['acq_date'].apply(lambda x: int(x[:4]))
datasetIncendios['month'] = datasetIncendios['acq_date'].apply(lambda x: int(x[5:7]))
datasetIncendios['day'] = datasetIncendios['acq_date'].apply(lambda x: int(x[8:10]))
datasetIncendios['year'] = datasetIncendios['acq_date'].apply(lambda x: int(x[:4]))

# Eliminar columna acq_date en ambos datasets
datasetIncendios = datasetIncendios.drop(['acq_date'], axis=1)
datasetRandomPoints = datasetRandomPoints.drop(['acq_date'], axis=1)

# Agregar una columna fire a cada dataset para diferenciar incendios de no-incendios
datasetIncendios['fire'] = 1
datasetRandomPoints['fire'] = 0

# Concatenar ambos datasets en uno solo y guardarlo como archivo csv
dataset = pd.concat([datasetIncendios, datasetRandomPoints], ignore_index=True)
dataset.to_csv('CompleteDataset.csv', index=False)